In [ ]:
!pip install pygooglenews --upgrade
!pip install transformers

In [ ]:
from pygooglenews import GoogleNews
from bs4 import BeautifulSoup
import pandas as pd
import requests
import math

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')

stop_words = stopwords.words('english')
stop_words.append('-')
stop_words.append('google')
stop_words.append('news')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
gn = GoogleNews()

s = gn.search('intitle:quetta gladiators vs karachi kings', from_='2019-03-10', to_='2019-03-12')

print(s['feed'].title)

"intitle:quetta gladiators vs karachi kings after:2019-03-10 before:2019-03-12" - Google News


In [ ]:
def words_for_images(title):
  new_lst = []
  required_img = []

  lst = title
  lst = lst.lower()
  lst = lst.split()
  #print(lst)

  #lst.replace('\"','')
  for word in lst:
    word = word.replace('\"','')
    word = word.replace('intitle:','')
    new_lst.append(word)
  
  for word in new_lst:
    if 'after' in word:
      new_lst.remove(word)

  for word in new_lst:  
    if 'before' in word:
      new_lst.remove(word)

  for word in new_lst:
    if word not in stop_words:
      required_img.append(word)

  #print(required_img)
  return required_img

In [ ]:
def required_images(images, required_img):
  images_lst = []
  for img_name in images:
    if any(x in str(img_name).lower() for x in required_img):
      images_lst.append(str(img_name))

  return images_lst

In [ ]:
# This function will return the number from 1 - 5. 1 will be extremely negative and 5 will be extremely positive

def sentiment_analysis(text):

  tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
  model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

  #tokens = tokenizer.encode(text, return_tensors='pt')

  num = math.ceil(len(text)/900)

  sent_score = []
  a = 0
  b = 900

  for iter in range(num):
    sub_text = text[a:b]
    #sub_tokens = tokens[a:b]
    tokens = tokenizer.encode(sub_text, return_tensors='pt')
    result = model(tokens)

    result_num = int(torch.argmax(result.logits)) + 1
    sent_score.append(result_num)

    a += 900
    b += 900
  #print(sent_score); exit()
  final_result = sum(sent_score)/len(sent_score)
  
  return final_result
  
  
  # result.logits   # To check the value of results

  #return int(torch.argmax(result.logits)) + 1

In [ ]:


total = len(s['entries'])

for tot in range(total):

  for a in range(len(s['entries'][tot].sub_articles)):
    full_text = []
    cleaned_text = []

    url = s['entries'][tot].sub_articles[a]['url']
    pub = s['entries'][tot].sub_articles[a]['publisher']
    title = s['entries'][tot].sub_articles[a]['title']
  
    page = requests.get(url)

    soup = BeautifulSoup(page.content, 'html.parser')

    #print(soup.body.text)
    for b in soup.body.stripped_strings:
      full_text.append(b)

    for c in full_text:
      if len(c)<130:
        continue
      else:
        cleaned_text.append(c)

    cleaned_text = ' '.join(cleaned_text)

    print(s['entries'][tot].sub_articles[a]['title'])

    images = soup.find_all('img')

    doc_title = s['feed'].title
    required_images_words = words_for_images(doc_title)

    our_images = required_images(images, required_images_words)

    # Calculating sentiment analysis
    sentiment_score = sentiment_analysis(cleaned_text)

    # Writing to the files
    file1 = open(title+'.txt', 'w')
    file1.write('url: '+url+'\n')
    file1.write('Publisher: '+pub + '\n')
    file1.write('Title: ' + title + '\n\n')

    file1.writelines(cleaned_text + '\n\n')
    file1.write('Sentiment Score: ' + str(sentiment_score) + '\n\n')
    file1.write('Images links \n')
    file1.writelines(our_images)
    file1.close()
    #break

Karachi Kings snatch victory away from Quetta Gladiators in dramatic clash on home turf
Karachi Kings beat Quetta Gladiators to reach playoffs
Karachi Kings beat Quetta Gladiators by 1 run in PSL Karachi Stadium thriller
PSL 2019: Quetta Gladiators coach Moin Khan envisions bright Pakistan future for Umar Akmal and Ahmed Shehzad


In [ ]:
sentiment_score

3.0

###Practice

In [ ]:
gn = GoogleNews(lang = 'en', country = 'PK')

In [ ]:
top_news = gn.top_news()

In [ ]:
top_news

In [ ]:
import math